<a href="https://colab.research.google.com/github/PeteH-89/GEOG5003M_Portfolio/blob/main/GEOG5003M_Portfolio_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOG5003M Portfolio assignment

Student ID number: 201790833



## Block One Submission
Using data introduced in the course create a visualisation showing the association between two variables. Using skills developed in the practical exercises take any steps required to clean or pre-process the data.  It is expected that you wills change the default appearance of the markers and include an appropriate title and axis labels. Further marks will be awarded for appropriate use of colour pallets, inclusion of additional categorical variables in the visualisation and other steps taken to ensure effective data visualisation.

Ensure all the code has detialed comments.

Write a brief paragraph in markdown (max. 300 words) detailing the visualisation choices made given the types of data you have chosen.

In [1]:
# import required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Data-preprocessing

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/OA_GEOG990M_data.csv'

In [ ]:
# Data visualisation

### Block One Submission
Edit this celll to include a brief paragraph (max. 300 words) detailing the visualisation choices made given the types of data you have chosen.











    

## Block Two Submission:

Using the course notebooks and lectures as a guide create a static visualisation showing the spatial distribution of a variable of your choice. The type of spatial visualisation chosen should be suitable for the data chosen and appropriate data cleaning and pre-processing steps undertaken. Using skills demonstrated in the python notebooks change the default appearance of the visualisation of choice. Include an appropriate title and legend labels. Further marks will be awarded for appropriate use of colour pallets and other customisations to aid interpretation and improve accessibility. Write a brief paragraph in markdown (max 300 words) detailing the visual design choices made.

In [ ]:
# import any further required packages

In [ ]:
# Data-preprocessing

In [ ]:
# Data Visualisation

### Block Two Submission
Edit this celll to include a brief paragraph (max. 300 words) detailing the spatial data visualisation choices made given the types of data you have chosen.







## Acknowledgements

The use of Gen AI must be acknowledged in an ‘Acknowledgements’ section of any piece of academic work where it has been used as a functional tool to assist in the process of creating academic work.
If it is suspected that you have used a Gen AI tool to produce part of your work, but you have not acknowledged this use, this could be investigated under the Academic Misconduct procedure.
The minimum requirement to include in acknowledgement:
- Name and version of the generative AI system used eg ChatGPT-4.0
- Publisher (company that made the AI system) eg OpenAI
- URL of the AI system
- Brief description (single sentence) of context in which the tool was used.

For example: “I acknowledge the use of ChatGPT-3.5 (Open AI, https://chat.openai.com/) to support development of code to generate legend headings and support formatting of figures”
In the code state:  Prompt: write some python code using the Geopandas package to add a legend title for a colour bar”

For example: “I acknowledge the use of Co-pilot (Microsoft https://chat.openai.com/ Accessed 2025-4-03) to support debugging errors as identified in the code, Prompt: Help debug this error {ERROR MESSAGE} genrated while using the geopandas package”